In [1]:
import geopandas
import os
from pygeotile.point import Point
from pygeotile.tile import Tile
from pygeotile.meta import TILE_SIZE
import requests
from PIL import Image
import random
from shutil import copy2
import json
import matplotlib.pyplot as plt
import fiona

## Collect image and prepare tiles

1. Use Nearmap's MapBrowser to collect the ortho image. In the paper we used the image from from Saturday 3 Dec 2022:
https://apps.nearmap.com/maps/#/@-37.8137625,144.9600780,15.90z,0d/V/20221203

2. Load the image into a GIS softwares (such as QGIS).
3. Clip the image with the shapefiles provided in the Github repo (with no_data value = 0)
4. Make VRT tiles out of the clipped raster.
5. Proceed with the codes below to prepare the training data.

## Labels from shapefile

In [3]:
polygons_paths = [os.path.join('./', 'CityPolygon', 'Buildings_MelbCBD_100m_buffer.shp')]

data_type = "MelbCBD_120cm" # Give name to you dataset
res = 1.2 # change resolution to 0.3 or 0.6 as needed
zoom = 17 # find the appropriate zoom levels for the corresponding resolution from here: https://help.nearmap.com/kb/articles/84-tile-api
rgb_path = os.path.join('./', 'Gather_tiles', data_type, 'image') # specify path for output


In [4]:
bf_gdf = 0
for i, polygons_path in enumerate(polygons_paths):
    if isinstance(bf_gdf, geopandas.geodataframe.GeoDataFrame):
        tmp_bf_gdf = geopandas.read_file(polygons_path)
        tmp_bf_gdf['file_id'] = i
        bf_gdf = bf_gdf.append(tmp_bf_gdf, ignore_index=True)
    else:
        bf_gdf = geopandas.read_file(polygons_path)
        bf_gdf['file_id'] = i

bf_gdf['bounds'] = bf_gdf.apply(lambda row: row['geometry'].bounds, axis=1)
bf_gdf['centroid'] = bf_gdf.apply(lambda row: row['geometry'].centroid, axis=1)
bf_gdf['i'] = bf_gdf.index
bf_gdf.head()

/Users/bneupane/miniforge3/envs/data-collect/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,structure_,property_i,footprint_,roof_type,tier,footprin_2,footprin_3,structur_2,structur_3,footprin_4,structur_4,geometry,file_id,bounds,centroid,i
0,808172.0,104783.0,Structure,Flat,1.0,33.0,17.5,37.0,17.5,15.5,15.5,"POLYGON ((321030.053 5813146.291, 321026.140 5...",0,"(321006.0486999638, 5813140.4046005625, 321042...",POINT (321023.730 5813159.409),0
1,812667.0,109953.0,Structure,Flat,1.0,30.5,17.5,30.5,17.5,13.5,13.0,"POLYGON ((321048.197 5813147.163, 321031.995 5...",0,"(321029.3472999659, 5813141.1014005635, 321048...",POINT (321038.201 5813151.339),1
2,804213.0,105686.0,Structure,Flat,1.0,26.0,18.0,26.0,18.0,8.0,8.0,"POLYGON ((321019.517 5813171.175, 321014.878 5...",0,"(321007.14169996674, 5813169.338500563, 321019...",POINT (321013.353 5813180.088),2
3,807603.0,105687.0,Structure,Gable,1.0,26.0,18.0,26.0,18.0,8.0,8.0,"POLYGON ((321014.878 5813169.339, 321010.838 5...",0,"(321003.10239996423, 5813167.739800559, 321014...",POINT (321008.990 5813178.312),3
4,818563.0,105685.0,Structure,Flat,1.0,26.5,18.0,32.5,18.0,9.0,8.5,"POLYGON ((321026.608 5813179.292, 321027.175 5...",0,"(321011.7606999666, 5813171.614300565, 321028....",POINT (321020.049 5813183.107),4


## Make label

In [101]:
#!/usr/bin/env python3
#  auto-rasterize
#
#     Huriel Reichel - huriel.ruan@gmail.com
#     Nils Hamel - nils.hamel@bluewin.ch
#     Copyright (c) 2020 Republic and Canton of Geneva
#
#  This program is free software: you can redistribute it and/or modify
#  it under the terms of the GNU General Public License as published by
#  the Free Software Foundation, either version 3 of the License, or
#  (at your option) any later version.
#
#  This program is distributed in the hope that it will be useful,
#  but WITHOUT ANY WARRANTY; without even the implied warranty of
#  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#  GNU General Public License for more details.
#
#  You should have received a copy of the GNU General Public License
#  along with this program.  If not, see <http://www.gnu.org/licenses/>.

from osgeo import gdal
from osgeo import ogr


def _rasterize(input, output, xmin, ymin, xmax, ymax, pixel):
    # input = "Datasets\\2020 Building Footprints\\shapefile_new.shp" 
    # output = "testt2.png"  
    # xmin = 314852
    # ymax = 5817273 
    # xmax = 323406
    # ymin = 5808587 
    # pixel = 1

    # Open the data source
    orig_data_source = ogr.Open(input)
    # Make a copy of the layer's data source because we'll need to 
    # modify its attributes table
    source_ds = ogr.GetDriverByName("Memory").CopyDataSource(
            orig_data_source, "")
    source_layer = source_ds.GetLayer(0)
    source_srs = source_layer.GetSpatialRef() 
    

    # Create the destination data source
    x_res = int((xmax - xmin) / pixel)
    y_res = int((ymax - ymin) / pixel)      
    
    target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res,
            y_res, 3, gdal.GDT_Byte)

    target_ds.SetGeoTransform((
            xmin, pixel, 0,
            ymax, 0, -pixel,
        ))
    target_ds.GetRasterBand(1).SetNoDataValue(0)
    target_ds.GetRasterBand(2).SetNoDataValue(0)
    target_ds.GetRasterBand(3).SetNoDataValue(0)
    if source_srs:
        # Make the target raster have the same projection as the source
        target_ds.SetProjection(source_srs.ExportToWkt())
    else:
        # Source has no projection (needs GDAL >= 1.7.0 to work)
        target_ds.SetProjection('LOCAL_CS["arbitrary"]')
    # Rasterize
    err = gdal.RasterizeLayer(target_ds, (3, 2, 1), source_layer,
            burn_values=(255, 255, 255))
    if err != 0:
        raise Exception("error rasterizing layer: %s" % err)

def rasterize(input, output, xmin, ymin, xmax, ymax, pixel):
    print('rasterizing based on reference raster')
    _rasterize(input, output, xmin, ymin, xmax, ymax, pixel)


In [102]:
tile_size = 256
image_path = 'path to the ortho image'+data_type+'/'

path, dirs, files = next(os.walk(image_path))
file_count = len(files)
images = []

for cnt in range(file_count):
    # Collect Images in an array
    collect = os.path.join(image_path,files[cnt])
    images.append(collect)
    print(files[cnt])
    
    tiff_src = gdal.Open(images[cnt])
    upper_left_x_coo, pixel_x_size, xskew, upper_left_y_coo, yskew, pixel_y_size = tiff_src.GetGeoTransform()
    pixel_y_size = pixel_x_size

    bottom_left_x_coo = upper_left_x_coo
    bottom_left_y_coo = upper_left_y_coo - (256 * pixel_y_size)
    sw = (bottom_left_x_coo, bottom_left_y_coo)
    
    upper_right_x_coo = upper_left_x_coo + (256 * pixel_x_size)
    upper_right_y_coo = upper_left_y_coo
    ne = (upper_right_x_coo, upper_right_y_coo)
    
    rasterize(polygons_paths[0], os.path.join('./', 'Gather_tiles', data_type,'label', str(files[cnt])[:-4]+'.png') , sw[0], sw[1], ne[0], ne[1], res)

MelbCBD_30cm_tiles.528.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.514.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.272.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.266.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.500.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.299.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.716.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.138.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.104.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.662.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.676.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.110.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.689.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.460.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.306.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.312.ti

MelbCBD_30cm_tiles.73.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.370.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.364.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.402.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.358.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.199.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.172.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.614.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.600.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.166.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.628.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.774.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.589.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.562.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.204.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.210.tif

MelbCBD_30cm_tiles.145.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.623.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.87.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.409.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.353.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.435.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.421.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.44.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.390.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.78.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.385.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.391.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.79.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.45.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.420.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.346.tif
ras

MelbCBD_30cm_tiles.534.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.508.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.278.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.522.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.244.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.536.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.293.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.654.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.683.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.456.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.330.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.324.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.442.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.495.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.494.tif
rasterizing based on reference raster
MelbCBD_30cm_tiles.325.ti

## Tiff Image to png

Use FIJI ImageJ to batch convert tiff to png

## Train and Test Split

In [125]:
data_type = "MelbCBD_120cm"
res = 1.2
res_name = str(120)
zoom = 17

""" Test and Train Splitter - RGB Images"""
image_path = os.path.join('./', 'Gather_tiles', data_type,'image')
image_files = [f for f in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, f))]
random.shuffle(image_files)
files = {}
number_of_all_files = len(image_files)
number_of_train_files = int(number_of_all_files*0.7)
files['train'] = image_files[:number_of_train_files]
files['test'] = image_files[number_of_train_files:]

#data_type = "Launceston-17"
file_map_path = "./training_data/" + data_type + "/file_map.json"
with open(file_map_path, 'w') as f:
  print("dumping")
  json.dump(files, f)

dumping


In [126]:
for k, list_of_files in files.items():
    counter = 0
    print("preparing ", k)
    out_path = os.path.join('./training_data/', data_type, k)
    for file in list_of_files:
        copy2(os.path.join(image_path, file), os.path.join(out_path, 'image', str(counter) + '_' + res_name + '.png'))
        copy2(os.path.join('./training_data/', data_type, 'label', file), os.path.join(out_path, 'label', str(counter) + '_' + res_name + '.png'))
        counter+=1

preparing  train
preparing  test
